In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
data=pd.read_csv("tmdb_5000_movies.csv")
data.head()
data.shape
data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [3]:
########데이터 전처리##########
#추천시스템 동작 : 기존에 이미 시청한 영화의 제목을 입력 -> 영화와 비슷한 영화를 추천
#                                                    컨텐츠기반추천
data=data[['id','genres', 'vote_average','vote_count', 'popularity', 
      'title', 'keywords', 'overview']]

# 오문희, 오케이마담 -> ??? 영화 추천  -> 추천알고리즘(컨텐츠/아이템기반)
#                                            연관규칙(apriori) -> 추천
#     장바구니분석(채소, 땅콩, 우유) -> ??? 스트로베리 추천
#  담배 -> 라이터 구매, 맥주 -> 쥐포
# insight추출 (기저귀 -> 맥주)



"""
필요로 하는 컬럼만 추출, 데이터 표준화/정규화,결측값 처리?
문자데이터(형태소분석, 불용어제거, 대/소문자 통일,...)
feature engineering(특성공학, )

xdata(수집데이터, 입력데이터, 독립변수) -> ydata(출력데이터, 종속변수)
국어, 영어, 수학, 언어능력(50점만점)   수능점수
80     90   100 ,  40   400점
         ...
100명 학생 점수 data를 수집

파생변수 : 기존의 독립변수로부터 연산을 수행하여 만든 변수

x변수들 -> 모델 -> y예측값(y hat)
90 100 95 -> ???예측

타이타닉 data 
승객명단          성별(파생)
Mrs. 이름...        0
Mr. 이름...           1
Dr. ...

"""

'\n필요로 하는 컬럼만 추출, 데이터 표준화/정규화,결측값 처리?\n문자데이터(형태소분석, 불용어제거, 대/소문자 통일,...)\nfeature engineering(특성공학, )\n\nxdata(수집데이터, 입력데이터, 독립변수) -> ydata(출력데이터, 종속변수)\n국어, 영어, 수학, 언어능력(50점만점)   수능점수\n80     90   100 ,  40   400점\n         ...\n100명 학생 점수 data를 수집\n\n파생변수 : 기존의 독립변수로부터 연산을 수행하여 만든 변수\n\nx변수들 -> 모델 -> y예측값(y hat)\n90 100 95 -> ???예측\n\n타이타닉 data \n승객명단          성별(파생)\nMrs. 이름...        0\nMr. 이름...           1\nDr. ...\n\n'

In [4]:
#data['vote_count'].quantile(0.9)#1838
#상위 10% 지점에 해당되는 vote_count 컬럼 값이 1838 이다.
tmp_m=data['vote_count'].quantile(0.89)#1683
data['vote_count']>=tmp_m
tmp_data=data.copy()[data['vote_count']>=tmp_m]
tmp_data.shape

m=data['vote_count'].quantile(0.9)
data=data[data['vote_count']>=m]
#vote_count값에 대해 대략 500등 안에 들어가는 데이터를 추출

#data.info()
#수치데이터(연속형(점수,온도), 범주형(성별,혈액형,학점))
#연속형 수치데이터 -> 구간 -> 범주화

In [5]:
data.shape
data.head()

,id,genres,vote_average,vote_count,popularity,title,keywords,overview
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."


In [6]:
c=data['vote_average'].mean()
c #6.96
m #1838

1838.4000000000015

In [7]:
#(WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C
def weighted_rating(x, m=m, c=c):
    v=x['vote_count']
    R=x['vote_average']
    return (v/(v+m)*R) + (m/(m+v)*c)
    
data['score']=data.apply(weighted_rating, axis=1)    

In [8]:
data.head()
data[['genres', 'keywords']].head()
#리스트 내부에 딕셔너리 구조로 저장

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."


In [77]:
#data['genres'].apply(ast.literal_eval)[0]

In [56]:
#eval()
listStr='[1,2,3,4,5]'
listStr

'[1,2,3,4,5]'

In [89]:
import ast
listData=ast.literal_eval(listStr)
listData

[1, 2, 3, 4, 5]

In [9]:
data['genres']=data['genres'].apply(ast.literal_eval)

NameError: name 'ast' is not defined

In [ ]:
data['keywords']=data['keywords'].apply(ast.literal_eval)

In [10]:
data[['genres', 'keywords']]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."
...,...,...
4291,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 9648, ""n...","[{""id"": 1366, ""name"": ""shotgun""}, {""id"": 13000..."
4300,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 53, ""name...","[{""id"": 2052, ""name"": ""traitor""}, {""id"": 6099,..."
4302,"[{""id"": 37, ""name"": ""Western""}]","[{""id"": 801, ""name"": ""bounty hunter""}, {""id"": ..."
4337,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name...","[{""id"": 422, ""name"": ""vietnam veteran""}, {""id""..."


In [33]:
data.iloc[0]['genres']
# 모든 데이터의 장르 이름을 출력
#     genres
# 0  Action
#    Adventure
#      ...
# 1  Adventure
#     ...

data.iloc[1]['keywords']
#apply함수 활용

[{'id': 270, 'name': 'ocean'},
 {'id': 726, 'name': 'drug abuse'},
 {'id': 911, 'name': 'exotic island'},
 {'id': 1319, 'name': 'east india trading company'},
 {'id': 2038, 'name': "love of one's life"},
 {'id': 2052, 'name': 'traitor'},
 {'id': 2580, 'name': 'shipwreck'},
 {'id': 2660, 'name': 'strong woman'},
 {'id': 3799, 'name': 'ship'},
 {'id': 5740, 'name': 'alliance'},
 {'id': 5941, 'name': 'calypso'},
 {'id': 6155, 'name': 'afterlife'},
 {'id': 6211, 'name': 'fighter'},
 {'id': 12988, 'name': 'pirate'},
 {'id': 157186, 'name': 'swashbuckler'},
 {'id': 179430, 'name': 'aftercreditsstinger'}]

In [44]:
data.iloc[0]['genres']

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [46]:
data['genres']

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4291                         [Horror, Mystery, Crime]
4300                                [Crime, Thriller]
4302                                        [Western]
4337                                   [Crime, Drama]
4602                                          [Drama]
Name: genres, Length: 481, dtype: object

In [11]:
data['genres']=data['genres'].apply(lambda x: [d['name'] for d in x]).apply(lambda x: " ".join(x))

TypeError: string indices must be integers

In [143]:
data['keywords']=data['keywords'].apply(lambda x: [d['name'] for d in x]).apply(lambda x: " ".join(x))

In [144]:
data.head(2)
data.to_csv("pre_tmdb_5000_movies.csv", index=False)

In [117]:
#########컨텐츠(성별,장르,토픽,예산,....) 기반 필터링 -> 추천시스템#########
#우리는 장르를 이용해서 추천시스템 만들 예정

In [145]:
#장르(문자열)->숫자로 변환 -> 벡터화 작업
from sklearn.feature_extraction.text import *
#scikit->sk
#scikit-learn.org

In [97]:
data.genres #벡터화

0       Action Adventure Fantasy Science Fiction
1                       Adventure Fantasy Action
2                         Action Adventure Crime
3                    Action Crime Drama Thriller
4               Action Adventure Science Fiction
                          ...                   
4291                        Horror Mystery Crime
4300                              Crime Thriller
4302                                     Western
4337                                 Crime Drama
4602                                       Drama
Name: genres, Length: 481, dtype: object

In [98]:
#1. fit_transform함수를 이용한 장르 벡터화
data.head(2)
#장르를 벡터화
count_vector=CountVectorizer(ngram_range=(1,3))
c_vector_genres=count_vector.fit_transform(data['genres'])

In [99]:
print(c_vector_genres.toarray())
print(c_vector_genres.shape)

[[1 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(481, 364)


In [100]:
#2. fit & transform함수를 이용한 장르 벡터화
count_vector=CountVectorizer(ngram_range=(1,3))
count_vector.fit(data['genres'])

CountVectorizer(ngram_range=(1, 3))

In [101]:
len(count_vector.get_feature_names())
count_vector.get_feature_names()
#          <-----------------------364---------------------------->
#          'action'   'action adventure' ... 'western drama adventure'
# 람보         1              1       0      0 ...        1
# ...
# 코만도       1              0       1      0            1
# ...
# 481편

# 모든 각각의 영화들은 364차원 공간에 점(point)으로 표시
# 점의 총 개수? 481개


#          발   손    공부  폭탄 여행
# 람보    1     2     0     3       0
# 코만도 1     2     0     3       0
# 토지    0    1      1     0       2

# 람보를 시청한 관객에게 어떤 영화를 추천?
# cos(람보,코만도)=14 / 14 = 1(똑같다)
# cos(람보,토지) = 2 / 9.xx=0.2xxx
# cos(람보,코만도)  >  cos(람보,토지) 
# => 람보를 본 사람에게 코만도를 추천하자!!!
# ...

['action',
 'action adventure',
 'action adventure animation',
 'action adventure comedy',
 'action adventure crime',
 'action adventure fantasy',
 'action adventure science',
 'action adventure thriller',
 'action adventure war',
 'action adventure western',
 'action comedy',
 'action comedy crime',
 'action comedy drama',
 'action comedy family',
 'action comedy science',
 'action crime',
 'action crime drama',
 'action crime mystery',
 'action crime thriller',
 'action drama',
 'action drama adventure',
 'action drama fantasy',
 'action drama history',
 'action drama horror',
 'action drama mystery',
 'action drama science',
 'action drama thriller',
 'action family',
 'action family fantasy',
 'action family romance',
 'action fantasy',
 'action fantasy comedy',
 'action horror',
 'action mystery',
 'action mystery science',
 'action mystery thriller',
 'action romance',
 'action science',
 'action science fiction',
 'action thriller',
 'action thriller adventure',
 'action thrille

In [70]:
# c_vector_genres.toarray().shape
# #(481, 364)
# #
# c_vector_genres.toarray()[3]

# #c_vector_genres.toarray()에서 481개 모든 영화들에 대한 쌍을 구성했을때
# #가장 코사인유사도가 큰 값을 출력(인덱스도 함께)
# #ex) 
# print(c_vector_genres.toarray()[0]) #d1
# print(c_vector_genres.toarray()[1]) #d2
# cos(d1,d2)=d1*d2 / d1요소값 제곱합에 대한 제곱근 * d2요소값 제곱합에 대한 제곱근
# ...
# cos(d1,d481)=d1*d481 / d1요소값 제곱합에 대한 제곱근 * d481요소값 제곱합에 대한 제곱근
# =>
# 최대값에 해당되는 인덱스? 그때의 코사인 유사도?
# data[최대값에 해당되는 인덱스][title] 출력 (추천영화)

In [71]:
data.iloc[0].genres

'Action Adventure Fantasy Science Fiction'

In [72]:
data.iloc[1].genres

'Adventure Fantasy Action'

Bow란 (Bag of Words, 단어가방): 문서를 구성하는 모든 단어들에 대해 표시하는 방법

문서1:나는 대한민국 사람
문서2:나는 미국 사람
문서3:나는 중국 사람

코퍼스(corpus, 말뭉치) : {나는, 대한민국, 미국, 중국, 사람}
코퍼스 : 단어(도메인) 사전

토지소설 -> 원고 -> 토지 코퍼스:{토지 소설에 등장하는 단어들}
태백산맥 소설 -> 원고 -> 태백산맥 단어들 중에는 토지 코퍼스에 있는 단어도 있지만,
없는 단어도 있을 것이다(예, '지리산' 단어는 토지에는 등장하지 않음 -> OOV라고 함)



Bow: 전체 문서{d1,...,dn}를 구성하는 단어장(vocabulary, {t1,t2,...,tm})를 만든 다음,
각각의 문서(n개)에 대해 단어장에 있는 각각의 단어가 문서 내에 포함되어 있는지,
되어 있지 않은지를 표시하는 방법


In [135]:
from sklearn.feature_extraction import DictVectorizer
#DictVectorizer:단어의 빈도 정보가 담긴 딕셔너리로부터  BOW 벡터를 생성
v=DictVectorizer(sparse=False)
D=[{'A':1, 'B':2},{'B':3,'C':1}]#단어의 빈도 정보가 담긴 딕셔너리
#1번 문서 : {'A':1, 'B':2}, 2번 문서 : {'B':3,'C':1}
x=v.fit_transform(D)
x #bow 출력
#sparse <-> dense
v.feature_names_
v.transform({'C':4, 'D':2})
#{'C':4, 'D':2}딕셔너리를 v변수에 저장된 벡터화 방식으로 변환(bow)해라


array([[0., 0., 4.]])

In [144]:
#CountVectorizer
#문서 -> 단어 토큰 리스트 -> 단어 출현 빈도 -> bow  벡터로 변환
corpus=[ #해당 도메인에 대한 관련 문서들의 집합
    'This is the first document',
    'This is the second second document',
    'And the third one',
    'The last document'
]
cv=CountVectorizer()
cv.fit(corpus)
print(cv.vocabulary_) #0~9번까지 총 10개의 단어로 구성됨
print(cv.transform(corpus).toarray())
#  0 1 2 3 4 5 6 7 8 9
# [0 1 1 1 0 0 0 1 0 1]
# 'This is the first document'

#새로운 문서를 입력받아 bow생성
cv.transform(['Something completely new']).toarray()
cv.transform(['This is the third doc']).toarray()
#3,7,8,9

{'this': 9, 'is': 3, 'the': 7, 'first': 2, 'document': 1, 'second': 6, 'and': 0, 'third': 8, 'one': 5, 'last': 4}
[[0 1 1 1 0 0 0 1 0 1]
 [0 1 0 1 0 0 2 1 0 1]
 [1 0 0 0 0 1 0 1 1 0]
 [0 1 0 0 1 0 0 1 0 0]]


array([[0, 0, 0, 1, 0, 0, 0, 1, 1, 1]], dtype=int64)

In [145]:
corpus=[ #해당 도메인에 대한 관련 문서들의 집합
    'This is the first document',
    'This is the second second document',
    'And the third one',
    'The last document'
]

In [147]:
cv=CountVectorizer(stop_words=['is','the'])
#stop_words:불용어()
cv.fit(corpus)
print(cv.vocabulary_)
print(cv.transform(corpus).toarray())

{'this': 7, 'first': 2, 'document': 1, 'second': 5, 'and': 0, 'third': 6, 'one': 4, 'last': 3}
[[0 1 1 0 0 0 0 1]
 [0 1 0 0 0 2 0 1]
 [1 0 0 0 1 0 1 0]
 [0 1 0 1 0 0 0 0]]


In [151]:
cv=CountVectorizer(stop_words='english')
cv.fit(corpus)
print(cv.vocabulary_)

{'document': 0, 'second': 1}


In [155]:
cv.get_stop_words()

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [156]:
cv=CountVectorizer(analyzer='char')
cv.fit(corpus)
cv.vocabulary_
# corpus=[ #해당 도메인에 대한 관련 문서들의 집합
#     '
# This is the first document',
#     'This is the second second document',
#     'And the third one',
#     'The last document']

{'t': 14,
 'h': 6,
 'i': 7,
 's': 13,
 ' ': 0,
 'e': 4,
 'f': 5,
 'r': 12,
 'd': 3,
 'o': 11,
 'c': 2,
 'u': 15,
 'm': 9,
 'n': 10,
 'a': 1,
 'l': 8}

In [157]:
cv=CountVectorizer(token_pattern="t\w+")
cv.fit(corpus)
cv.vocabulary_

{'this': 2, 'the': 0, 'third': 1}

In [ ]:
#n-그램(Word2Vec):단어의 크기, 
#1(모노)-그램:토큰 하나만 단어, 2(바이)-그램:두개 토큰을 하나의 단어로 사용
#This is the first document 문서를 2(바이)-그램으로 단어를 표현
#This is, is the, the first, first document 


In [158]:
cv=CountVectorizer(ngram_range=(2,2))
cv.fit(corpus)
cv.vocabulary_
# This is the first document',
#     'This is the second second document',
#     'And the third one',
#     'The last document']

{'this is': 11,
 'is the': 2,
 'the first': 6,
 'first document': 1,
 'the second': 8,
 'second second': 5,
 'second document': 4,
 'and the': 0,
 'the third': 9,
 'third one': 10,
 'the last': 7,
 'last document': 3}

In [159]:
cv=CountVectorizer(ngram_range=(1,2))
cv.fit(corpus)
cv.vocabulary_

{'this': 20,
 'is': 5,
 'the': 13,
 'first': 3,
 'document': 2,
 'this is': 21,
 'is the': 6,
 'the first': 14,
 'first document': 4,
 'second': 10,
 'the second': 16,
 'second second': 12,
 'second document': 11,
 'and': 0,
 'third': 18,
 'one': 9,
 'and the': 1,
 'the third': 17,
 'third one': 19,
 'last': 7,
 'the last': 15,
 'last document': 8}

In [161]:
cv=CountVectorizer(ngram_range=(1,2), token_pattern="t\w+")
cv.fit(corpus)
cv.vocabulary_

{'this': 3, 'the': 0, 'this the': 4, 'third': 2, 'the third': 1}

In [175]:
corpus=['This is the first document',
    'This is the second second second second second document',
    'And the third one',
    'The last document']

#빈도수 기반 
cv=CountVectorizer(min_df=2)
cv.fit(corpus)
cv.vocabulary_

{'this': 3, 'is': 1, 'the': 2, 'document': 0}

In [178]:
cv.transform(corpus).toarray()
#cv.transform(corpus).toarray().sum(axis=0) #[3, 2, 4, 2]

array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 0, 1, 0],
       [1, 0, 1, 0]], dtype=int64)

In [121]:
#eval함수:파이썬 내장함수
eval("(5*2)/4")
eval("max([1,2,3])")
#eval함수는 보안에 취약 -> 보안에 강한 함수 필요성 -> literal_eval함수 등장
s={'a':3,'b':5}
s['a']


3

In [122]:
s2="{'a':3,'b':5}"
ast.literal_eval(s2)['a']

3

In [102]:
# eval("10*2")
# ast.literal_eval("10*2")
#literal_eval함수는 파이썬 기본 자료형에 대해서만 지원(매우 엄격)
#간단한 수식등은 eval 함수 사용

In [146]:
from sklearn.metrics.pairwise import cosine_similarity

In [147]:
c_vector_genres #영화 : 481 x 장르 종류 : 364
# 영화 : 481편에 대한 코사인 유사도(481*481)
# 영화1, 영화2
# ...
# 영화1, 영화481
# 영화2, 영화1
# ...
# 영화2, 영화481
# ...
# 영화481, 영화480
cosine_similarity(c_vector_genres, c_vector_genres).shape
cosine_similarity(c_vector_genres, c_vector_genres)

array([[1.        , 0.47140452, 0.35355339, ..., 0.        , 0.        ,
        0.        ],
       [0.47140452, 1.        , 0.33333333, ..., 0.        , 0.        ,
        0.        ],
       [0.35355339, 0.33333333, 1.        , ..., 0.        , 0.23570226,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.23570226, ..., 0.        , 1.        ,
        0.57735027],
       [0.        , 0.        , 0.        , ..., 0.        , 0.57735027,
        1.        ]])

In [161]:
cosine_similarity(c_vector_genres, c_vector_genres)[0]
#코사인 유사도를 구합니다(481*481)


genre_c_sim=cosine_similarity(c_vector_genres, c_vector_genres).argsort()[:,::-1]
# 각 영화별 코사인 유사도 값을 기준으로 내림차순 정렬

In [184]:
#"The Dark Knight Rises"영화가 코사인 유사도가 가장 높은 영화를 리턴
def get_recommend_movie_list(df, movie_title, top=30):
        target_movie_index=df[df['title']==movie_title].index.values #[3]   
        sim_index=genre_c_sim[target_movie_index, :top].reshape(-1)
        sim_index=sim_index[sim_index!=target_movie_index]
        res=df.iloc[sim_index].sort_values('score', ascending=False)[:10]
        return res


In [185]:
get_recommend_movie_list(data, movie_title="The Dark Knight Rises")


,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
65,155,Drama Action Crime Thriller,8.2,12002,187.322927,The Dark Knight,dc comics crime fighter secret identity scarec...,Batman raises the stakes in his war on crime. ...,8.035690
2091,274,Crime Drama Thriller,8.1,4443,18.174804,The Silence of the Lambs,based on novel psychopath horror suspense seri...,"FBI trainee, Clarice Starling ventures into a ...",7.767228
2760,264644,Drama Thriller,8.1,2757,66.113340,Room,based on novel carpet isolation kidnapping imp...,Jack is a young boy of 5 years old who has liv...,7.645138
351,1422,Drama Thriller Crime,7.9,4339,63.429157,The Departed,undercover boston police friends mafia underco...,"To take down South Boston's Irish Mafia, the p...",7.621146
1850,111,Action Crime Drama Thriller,8.0,2948,70.105981,Scarface,miami corruption capitalism cuba prohibition b...,After getting a green card in exchange for ass...,7.601698
4337,103,Crime Drama,8.0,2535,58.845025,Taxi Driver,vietnam veteran taxi obsession drug dealer nig...,A mentally unstable Vietnam War veteran works ...,7.564085
1051,146233,Drama Thriller Crime,7.9,3085,88.496873,Prisoners,pennsylvania kidnapping maze vigilante rural s...,When Keller Dover's daughter and her friend go...,7.550121
828,24,Action Crime,7.7,4949,79.754966,Kill Bill: Vol. 1,japan coma martial arts kung fu underworld yak...,An assassin is shot at the altar by her ruthle...,7.500378
3701,641,Crime Drama,7.9,2443,11.573034,Requiem for a Dream,drug addiction junkie heroin speed diet unsoci...,The hopes and dreams of four ambitious people ...,7.497657
1829,6977,Crime Drama Thriller,7.7,3003,53.645267,No Country for Old Men,texas drug traffic hitman united states–mexico...,"Llewelyn Moss stumbles upon dead bodies, $2 mi...",7.420140


In [113]:
x=np.array([20,10,30,13])
#x.argsort() #x를 오름차순정렬 10 13 20 30, array([1, 3, 0, 2], dtype=int64)
np.sort(x)[::-1]

array([30, 20, 13, 10])

In [114]:
np.argsort(x)

array([1, 3, 0, 2], dtype=int64)

In [116]:
np.argsort(-x)

array([2, 0, 3, 1], dtype=int64)

In [115]:
x[np.argsort(x)]

array([10, 13, 20, 30])

In [117]:
x[np.argsort(-x)]

array([30, 20, 13, 10])

In [120]:
data=np.random.randn(4,3)

In [121]:
data

array([[ 1.26878933,  0.84399893,  1.49138481],
       [-1.24719335,  0.62508265, -0.29466095],
       [-0.03850067,  0.62366811,  0.19085082],
       [-2.09962809,  0.89462401, -0.79716152]])

In [130]:
#columns index(0, 1, 2) 별로 정렬

#0번 컬럼 인덱스를 기준으로 내림차순 정렬
print(data[:,0])
print(data[:,0].argsort()) #[3, 1, 2, 0]
print(data[:,0].argsort()[::-1]) #[0 2 1 3]
print(data[data[:,0].argsort()[::-1]])

print("="*50)

#1번 컬럼 인덱스를 기준으로 내림차순 정렬
print(data[:,1])
print(data[:,1].argsort()) #[3, 1, 2, 0]
print(data[:,1].argsort()[::-1]) #[0 2 1 3]
print(data[data[:,1].argsort()[::-1]])

print("="*50)

#2번 컬럼 인덱스를 기준으로 내림차순 정렬
print(data[:,2])
print(data[:,2].argsort()) #[3, 1, 2, 0]
print(data[:,2].argsort()[::-1]) #[0 2 1 3]
print(data[data[:,2].argsort()[:,::-1]])

[ 1.26878933 -1.24719335 -0.03850067 -2.09962809]
[3 1 2 0]
[0 2 1 3]
[[ 1.26878933  0.84399893  1.49138481]
 [-0.03850067  0.62366811  0.19085082]
 [-1.24719335  0.62508265 -0.29466095]
 [-2.09962809  0.89462401 -0.79716152]]
[0.84399893 0.62508265 0.62366811 0.89462401]
[2 1 0 3]
[3 0 1 2]
[[-2.09962809  0.89462401 -0.79716152]
 [ 1.26878933  0.84399893  1.49138481]
 [-1.24719335  0.62508265 -0.29466095]
 [-0.03850067  0.62366811  0.19085082]]
[ 1.49138481 -0.29466095  0.19085082 -0.79716152]
[3 1 2 0]
[0 2 1 3]
[[ 1.26878933  0.84399893  1.49138481]
 [-0.03850067  0.62366811  0.19085082]
 [-1.24719335  0.62508265 -0.29466095]
 [-2.09962809  0.89462401 -0.79716152]]


In [186]:
###########아이템 기반 협업 필터링##########
#사용자의 행동 패턴을 기반으로 추천

In [190]:
data=pd.read_csv("ratings_small.csv")
data.head()
#아이템 기반 협업 필터링 : user-item 형식의 테이블로 변환
#테이블 변환시 자주 사용되는 함수 : pivot_table
data=data.pivot_table('rating', index='userId', columns='movieId') #rating:테이블 데이터
print(data.head())
"""
movieId   1   2  3  4 ... xxxxxxx
userId
1              4    2          
2               ...
3        3     2 ...
..
yyyyyy
"""

movieId  1       2       3       4       5       6       7       8       \
userId                                                                    
1           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           NaN     NaN     4.0     NaN     NaN     NaN     NaN     NaN   

movieId  9       10      ...  161084  161155  161594  161830  161918  161944  \
userId                   ...                                                   
1           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     4.0  ...     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN  ...     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     4.0  ...     NaN     NaN     NaN     NaN     NaN     N

'\nmovieId   1   2  3  4 ... xxxxxxx\nuserId\n1              4    2          \n2               ...\n3        3     2 ...\n..\nyyyyyy\n'

In [191]:
data.shape #(671, 9066)

(671, 9066)

In [12]:
#영화 아이디는 있지만 제목이 없어서 어떤 영화인지 알 수 없음
movies=pd.read_csv("tmdb_5000_movies.csv")

In [17]:
movies.head()
movies.rename(columns={'id':'movieId'}, inplace=True)
movies



# 1.예산
# -예산 vs 인기도(x:예산,y:인기도)
# -1 <=상관계수:공분산(x,y) / Sx*Sy< 1

# 2.장르
# -장르 vs 인기도
# -장르 vs 점수
# -장르별로 인기도와 점수를 기준으로 비슷한 영화 묶음(유클리디안, 코사인유사도)

# 3. 키워드
# -키워드와 매출의 관계?

# 4. 줄거리(개요)
# -자연어처리(단어 분리 -> 불용어 제거... -> 줄거리가 비슷한(공통 단어) 영화 추천)
# -줄거리에 등장하는 단어와 인기도 사이의 관계

# 5. 출시일
# -월별/분기별/연도별 영화 매출액 변화
# -계절과 장르의 관계?( groupby 계절 -> 각 그룹에 대해 장르 별 출시 영화 개수 count)
# ->시각화 
# ...
# 6. runtime
# 장르별 runtime평균
# runtime - 흥행(수익, 관객수, 평점) 관계?

# ...
    

,budget,genres,homepage,movieId,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"

In [18]:
ratings=pd.read_csv("ratings_small.csv")
ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [19]:
movies.columns

Index(['budget', 'genres', 'homepage', 'movieId', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count'],
      dtype='object')

In [31]:
ratings_movies=pd.merge(ratings, movies, on='movieId')
ratings_movies.head(10)
#userid 가 1인 사람의 모든 자료를 출력
ratings_movies[ratings_movies['userId']==1]


,userId,movieId,rating,timestamp,budget,genres,homepage,keywords,original_language,original_title,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,1,2105,4.0,1260759139,11000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,"[{""id"": 3687, ""name"": ""graduation""}, {""id"": 61...",en,American Pie,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1999-07-09,235483004,95.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,There's nothing like your first piece.,American Pie,6.4,2296
47,1,2294,2.0,1260759108,22000000,"[{""id"": 35, ""name"": ""Comedy""}]",NaN,"[{""id"": 886, ""name"": ""film making""}, {""id"": 16...",en,Jay and Silent Bob Strike Back,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2001-08-22,33788161,104.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Hollywood had it coming,Jay and Silent Bob Strike Back,6.4,480


In [20]:
ratings_movies.shape

(18571, 23)

In [ ]:
# cos(홍길동,임꺽정), cos(홍길동,이순신), cos(이순신,임꺽정)
#             a    b      c     d       e
# 홍길동      1           5     4
# 임꺽정      5    4      4     4       2
# 이순신      1           3     3       4

# cos(a,b),cos(a,c),cos(a,d),cos(a,e),...cos(d,e)
#    홍길동   임꺽정   이순신
# a    1         5          1
# b              4
# c    5         4          3
# d    4         4          3
# e              2          4 


In [25]:
data=ratings_movies.pivot_table('rating', index='userId', columns="title").fillna(0)
data.shape
data.head()
#사용자별 영화 제목에 따른 평점 정보 확인 -> 유사한 사용자 검출(코사인 유사도 이용)

title,10 Things I Hate About You,12 Angry Men,1408,15 Minutes,16 Blocks,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2046,21 Grams,25th Hour,...,Willy Wonka & the Chocolate Factory,World Trade Center,X-Men Origins: Wolverine,Y Tu Mamá También,You Only Live Twice,"You, Me and Dupree",Young Frankenstein,Zodiac,eXistenZ,xXx
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
data=data.transpose()
data

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15 Minutes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16 Blocks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"You, Me and Dupree",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Young Frankenstein,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
Zodiac,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
movie_sim=cosine_similarity(data,data)

In [41]:
data.index

Index(['10 Things I Hate About You', '12 Angry Men', '1408', '15 Minutes',
       '16 Blocks', '20,000 Leagues Under the Sea', '2001: A Space Odyssey',
       '2046', '21 Grams', '25th Hour',
       ...
       'Willy Wonka & the Chocolate Factory', 'World Trade Center',
       'X-Men Origins: Wolverine', 'Y Tu Mamá También', 'You Only Live Twice',
       'You, Me and Dupree', 'Young Frankenstein', 'Zodiac', 'eXistenZ',
       'xXx'],
      dtype='object', name='title', length=856)

In [44]:
movie_sim #array
movie_sim_df=pd.DataFrame(movie_sim,
            index=data.index,
            columns=data.index)   #array -> dataframe

In [45]:
movie_sim_df.head(3)

title,10 Things I Hate About You,12 Angry Men,1408,15 Minutes,16 Blocks,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2046,21 Grams,25th Hour,...,Willy Wonka & the Chocolate Factory,World Trade Center,X-Men Origins: Wolverine,Y Tu Mamá También,You Only Live Twice,"You, Me and Dupree",Young Frankenstein,Zodiac,eXistenZ,xXx
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You,1.0,0.0,0.0,0.182153,0.0,0.022069,0.085323,0.0,0.0,0.10349,...,0.059856,0.0,0.161801,0.088076,0.0,0.0,0.097588,0.0,0.0,0.014121
12 Angry Men,0.0,1.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.00000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
1408,0.0,0.0,1.0,0.447214,0.0,0.173381,0.028245,0.0,0.0,0.00000,...,0.146955,0.0,0.148968,0.140265,0.0,0.0,0.191675,0.0,0.0,0.000000


In [46]:
movie_sim_df.index

Index(['10 Things I Hate About You', '12 Angry Men', '1408', '15 Minutes',
       '16 Blocks', '20,000 Leagues Under the Sea', '2001: A Space Odyssey',
       '2046', '21 Grams', '25th Hour',
       ...
       'Willy Wonka & the Chocolate Factory', 'World Trade Center',
       'X-Men Origins: Wolverine', 'Y Tu Mamá También', 'You Only Live Twice',
       'You, Me and Dupree', 'Young Frankenstein', 'Zodiac', 'eXistenZ',
       'xXx'],
      dtype='object', name='title', length=856)

In [52]:
movie_sim_df['X-Men Origins: Wolverine'].sort_values(ascending=False)[1:11]

title
Romeo Must Die                        0.649625
The Wedding Planner                   0.631669
Dogtown and Z-Boys                    0.501189
An Unfinished Life                    0.485643
Conquest of the Planet of the Apes    0.474626
Reign Over Me                         0.458155
The Terminal                          0.445337
Young Frankenstein                    0.423840
Whale Rider                           0.394136
Madagascar                            0.392497
Name: X-Men Origins: Wolverine, dtype: float64

In [ ]:
#ratings.csv와 movies.csv 두 파일을 읽어서 추천시스템 제작

#1. 아이템 기반 추천
#2. 사용자 기반 추천

# 아이템 기반 추천 실행 예)
# 재밌게 봤던 영화 제목 입력 : Goodfellas 입력시 'Goodfellas' 영화가 코사인 유사도(평점)가 
# 가장 높은 영화를 추천...

# 사용자 기반 추천  실행 예)   
# 사용자 아이디 입력 : 1
# 사용자 아이디가 1인 사람과 유사한 사용자(들)을 검색하여 영화를 추천

